# 本地控制代理 (Local Control Agent)

`本地控制代理 (LocalControlAgent)` 是多代理系统（MAS）中的一个关键组件。它充当特定控制算法（如PID控制器）的封装器，通过处理 `MessageBus` (消息总线) 上的通信，使其能够在更广泛的系统中运行。其主要职责是监听传感器数据，使用其封装的控制器进行处理，并发布一个控制动作。

## 核心概念

- **封装 (Encapsulation):** 它将控制逻辑的复杂性隐藏在一个专用的 `Controller` 对象内部。
- **事件驱动 (Event-Driven):** 该代理不是在连续循环中运行。相反，它对消息作出反应。其核心逻辑在其订阅的 `observation_topic` 上有新消息到达时被触发。
- **通信 (Communication):** 它使用 `MessageBus` 来订阅主题（用于传感器数据和命令）和发布主题（用于控制动作）。

## 关键参数

- `agent_id`: 代理的唯一标识符。
- `controller`: 一个实现了 `Controller` 接口的类的实例。
- `message_bus`: 系统的共享消息总线。
- `observation_topic`: 代理监听传感器读数的主题。
- `observation_key`: 观测消息中包含过程变量的特定键（例如，'water_level'）。
- `action_topic`: 代理发布其计算出的控制信号的主题。

## 仿真示例

以下示例演示了如何设置和使用 `LocalControlAgent`。我们创建了一个模拟控制器、一个消息总线和代理本身。然后，我们模拟一个传感器发布新的测量值，我们将看到代理如何处理这些信息并发布一个相应的控制动作，该动作由一个模拟的执行器接收。

**注意:** 要运行此Notebook，您需要从项目的根目录运行Jupyter服务器。

In [ ]:
import time
from swp.core.interfaces import Agent, Controller, State
from swp.central_coordination.collaboration.message_bus import MessageBus, Message
from swp.local_agents.control.local_control_agent import LocalControlAgent

# 1. 用于演示的模拟比例控制器
class MockProportionalController(Controller):
    def __init__(self, gain):
        self.gain = gain
        self.setpoint = 5.0 # 目标值
        
    def compute_control_action(self, state: State, dt: float) -> any:
        error = self.setpoint - state['process_variable']
        return self.gain * error

    def set_setpoint(self, new_setpoint: float):
        self.setpoint = new_setpoint

# 2. 设置通信总线
bus = MessageBus()

# 3. 模拟执行器以接收控制信号
received_actions = []
def actuator_callback(message: Message):
    print(f"执行器收到: {message}")
    received_actions.append(message)

bus.subscribe('gate_control_topic', actuator_callback)

# 4. 实例化控制器和代理
p_controller = MockProportionalController(gain=0.5)
control_agent = LocalControlAgent(
    agent_id="gate_controller_1",
    controller=p_controller,
    message_bus=bus,
    observation_topic='reservoir_level_topic',
    observation_key='water_level',
    action_topic='gate_control_topic',
    dt=1.0
)

# 5. 模拟传感器发布数据
print("--- 传感器发布数据 ---")
sensor_data = {'water_level': 4.8, 'timestamp': time.time()}
bus.publish('reservoir_level_topic', sensor_data)

# 消息总线会立即处理消息
print("--- 验证 ---")
assert len(received_actions) == 1
# 修正了浮点数比较的断言
assert abs(received_actions[0]['control_signal'] - (0.5 * (5.0 - 4.8))) < 1e-9
print("成功接收到正确的控制信号。")